# Triple Pendulum CHAOS!!!

From this tweet earlier this week:

<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">Slightly different initial conditions have this effect on a non-linear chaotic system like a triple pendulum: see how 41 of them do perform <a href="https://t.co/dP00jSKb5l">pic.twitter.com/dP00jSKb5l</a></p>&mdash; Massimo (@Rainmaker1973) <a href="https://twitter.com/Rainmaker1973/status/838462016532668418">March 5, 2017</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

My blog post with [Double Pendulum](https://jakevdp.github.io/blog/2012/08/18/matplotlib-animation-tutorial/) code.

Here's the first of the three relevant equations from the appendix of [this document](https://www.nickeyre.com/images/triplependulum.pdf):

![Equation Screenshot](../../images/eqn-screenshot.png)

Yikes.

Adapted from the paper [Gede et al. 2013](https://www.researchgate.net/publication/267490975_Constrained_Multibody_Dynamics_With_Python_From_Symbolic_Equation_Generation_to_Publication)

It uses the [Kane's Method](http://docs.sympy.org/dev/modules/physics/mechanics/kane.html) implementation within the [Sympy package](http://sympy.org)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from sympy import symbols
from sympy.physics import mechanics as mech

from sympy import Dummy, lambdify
from scipy.integrate import odeint

In [ ]:
def integrate_pendulum(n, times,
                       initial_positions=135,
                       initial_velocities=0,
                       lengths=None, masses=1):
    """Integrate a multi-pendulum with `n` sections"""
    #-------------------------------------------------
    # Step 1: construct the pendulum model
    
    # Generalized coordinates and velocities
    q = mech.dynamicsymbols('q:{0}'.format(n))
    u = mech.dynamicsymbols('u:{0}'.format(n))

    # mass and length
    m = symbols('m:{0}'.format(n))
    l = symbols('l:{0}'.format(n))

    # gravity and time symbols
    g, t = symbols('g,t')
    
    #--------------------------------------------------
    # Step 2: build the model using the Kane Method

    # Create pivot point reference frame
    A = mech.ReferenceFrame('A')
    P = mech.Point('P')
    P.set_vel(A, 0)

    # lists to hold reference frames, particles, and forces
    # for each pendulum in the chain
    frames = [] 
    particles = []
    forces = []
    kin_odes = []

    for i in range(n):
        # Create a new reference frame 
        Ai = A.orientnew('A' + str(i), 'Axis', [q[i], A.z])
        Ai.set_ang_vel(A, u[i] * A.z)
        frames.append(Ai) # Add it to Frames list

        # Create a point in this reference frame
        Pi = P.locatenew('P' + str(i), l[i] * Ai.x)
        Pi.v2pt_theory(P, A, Ai) # Set velocity of Pi

        # Create a new particle at this point
        Pai = mech.Particle('Pa' + str(i), Pi, m[i])
        particles.append(Pai)

        # Set forces & compute kinematic ODE
        forces.append((Pi, m[i] * g * A.x))
        kin_odes.append(q[i].diff(t) - u[i])

        P = Pi

    # Generate equations of motion
    kane = mech.KanesMethod(A, q_ind=q, u_ind=u,
                            kd_eqs=kin_odes)
    fr, frstar = kane.kanes_equations(forces, particles)
    
    #-----------------------------------------------------
    # Step 3: numerically evaluate equations and integrate

    # initial positions and velocities – assumed to be given in degrees
    y0 = np.concatenate([np.deg2rad(initial_positions) * np.ones(n),
                         np.deg2rad(initial_velocities) * np.ones(n)])
    y0 = np.asarray(y0)
        
    # lengths and masses
    if lengths is None:
        lengths = np.ones(n) / n
    lengths *= np.ones(n)
    masses *= np.ones(n)

    # Fixed parameters: gravitational constant, masses, and lengths
    parameters = [g] # Parameter Definitions
    parameter_vals = [9.81] # First we define gravity

    for i in range(n):
        parameters += [l[i], m[i]]
        parameter_vals += [lengths[i], masses[i]]

    # dummy symbols for translating to functions of time
    dummy_symbols = [Dummy() for i in q + u]
    dummy_dict = dict(zip(q + u, dummy_symbols))
    kds = kane.kindiffdict()

    # substitute dummy symbols for qdot terms
    mm_sym = kane.mass_matrix_full.subs(kds).subs(dummy_dict)
    fo_sym = kane.forcing_full.subs(kds).subs(dummy_dict)

    # functions for numerical calculation 
    mm_func = lambdify(dummy_symbols + parameters, mm_sym)
    fo_func = lambdify(dummy_symbols + parameters, fo_sym)

    # function which computes the derivatives of parameters
    def rhs(y, t, args):
        vals = np.hstack((y, args))
        sol = np.linalg.solve(mm_func(*vals), fo_func(*vals))
        return np.array(sol).T[0]

    # ODE integration
    return odeint(rhs, y0, times, args=(parameter_vals,))

In [ ]:
def get_xy_position(p, lengths=None):
    p = np.atleast_2d(p)
    n = p.shape[1] // 2
    if lengths is None:
        lengths = np.ones(n) / n
    zeros = np.zeros(p.shape[0])[:, None]
    x = np.hstack([zeros, lengths * np.sin(p[:, :n])])
    y = np.hstack([zeros, -lengths * np.cos(p[:, :n])])
    return np.cumsum(x, 1), np.cumsum(y, 1)

In [ ]:
t = np.linspace(0, 12, 240)
p = integrate_pendulum(n=2, times=t)

x, y = get_xy_position(p)
plt.plot(x, y);

In [ ]:
from matplotlib import animation
from IPython.display import HTML

def animate_pendulum(n, **kwargs):
    t = np.linspace(0, 10, 200)
    p = integrate_pendulum(n, t)
    x, y = get_xy_position(p)
    
    fig, ax = plt.subplots(figsize=(6, 6))
    fig.subplots_adjust(left=0.01, right=0.99, bottom=0.01, top=0.99)
    ax.grid()
    ax.axis('equal')
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)

    line, = ax.plot([], [], 'o-', lw=2)

    def init():
        line.set_data([], [])
        return line,

    def animate(i):
        line.set_data(x[i], y[i])
        return line,

    ani = animation.FuncAnimation(fig, animate, frames=len(t),
                                  interval=1000 * t.max() / len(t),
                                  blit=True, init_func=init)
    plt.close(fig)
    return HTML(ani.to_html5_video())
    
animate_pendulum(3)

In [ ]:
from matplotlib import collections


def animate_pendulum_multiple(n, n_pendulums=41, jitter=1E-5, track_length=15,
                              filename=None, **kwargs):
    oversample = 3
    track_length *= oversample
    
    t = np.linspace(0, 10, oversample * 200)
    p = [integrate_pendulum(n, t, initial_positions=135 + i * jitter / n_pendulums)
         for i in range(n_pendulums)]
    positions = np.array([get_xy_position(pi) for pi in p])
    positions = positions.transpose(0, 2, 3, 1)
    # positions is a 4D array: (npendulums, len(t), n+1, xy)
    
    fig, ax = plt.subplots(figsize=(6, 6))
    fig.subplots_adjust(left=0.01, right=0.99, bottom=0.01, top=0.99)
    ax.grid()
    ax.axis('equal')
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    
    track_segments = np.zeros((n_pendulums, 0, 2))
    tracks = collections.LineCollection(track_segments, cmap='rainbow')
    tracks.set_array(np.linspace(0, 1, n_pendulums))
    ax.add_collection(tracks)
    
    pendulum_segments = np.zeros((n_pendulums, 0, 2))
    pendulums = collections.LineCollection(pendulum_segments, colors='black')
    ax.add_collection(pendulums)

    def init():
        pendulums.set_segments(np.zeros((n_pendulums, 0, 2)))
        tracks.set_segments(np.zeros((n_pendulums, 0, 2)))
        return pendulums, tracks

    def animate(i):
        i = i * oversample
        pendulums.set_segments(positions[:, i])
        sl = slice(max(0, i - track_length), i)
        tracks.set_segments(positions[:, sl, -1])
        return pendulums, tracks

    interval = 1000 * oversample * t.max() / len(t)
    ani = animation.FuncAnimation(fig, animate, frames=len(t) // oversample,
                                  interval=interval,
                                  blit=True, init_func=init)
    
    if filename:
        fps = 1000 / interval
        if filename.endswith('gif'):
            ani.save(filename, fps=fps, writer='imagemagick')
        elif filename.endswith('mp4'):
            ani.save(filename, fps=fps, extra_args=['-vcodec', 'libx264'])
        else:
            ani.save(filename, fps=fps)
    plt.close(fig)

    return HTML(ani.to_html5_video())
    
animate_pendulum_multiple(3, filename='triple-pendulum.mp4')